# HW 3-Part-of-Speech Tagging with HMMs + Decoding Techniques (Greedy and Viterbi)

- Detravious Jamari Brinkley
- CSCI-544: Applied Natural Language Processing
- python version: 3

---

1. Part-of-Speech (POS) Tagging [a type of sequence labelling task where of a given word, assign the part of speech]
2. HMMs (Hidden Markov Model) [a generative-based model that's used for POS Tagging]
    1. Generative-based [provides the probabilities for all possible combinations of values of variables in the set using the joint distribution]
    2. With POS Tagging: Given a sequence of observations (sentences), the task is to infer the most likely sequence of hidden states (POS Tags) that could have generated the observed data.
3. **Decoding Techniques:**
    1. Greedy [find the optimal (OPT) solution at each step]
    2. Viterbi [make use of dynammic programming to find the OPT solution with backtracking while searching the entire search space]
4. **Notes of the data and given files:**
    - Dataset: Wall Street Journal section of the Penn Treebank
    - Folder named `data` with the following files:
        1. `train`, sentences *with* human-annotated POS Tags
        2. `dev`, sentences *with* human-annotated POS Tags
        3. `test`, sentences *without* POS Tags, thus predict the POS Tags
    - Format: Blank like at the end of each sentence. Each line contains 3 items separated by the `\t`, the tab symbol. These three items are
        1. Index of the word in the sentence
        2. Word type
        3. POS Tag



In [1]:
import sys

import numpy as np
import pandas as pd

from tqdm import tqdm
from collections import defaultdict

/var/folders/fz/zn5r8vq12nv5p23dtlr15sk40000gn/T/ipykernel_64887/1197507032.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Load and Update Data
- [x] Find a way to separate sentences when loading the df.

In [2]:
def load_data(file_path: str, file_name: str, is_test_file: bool, config_index: bool = True):
    
    if config_index == True:
        if is_test_file != True:
            file =  file_path + file_name
            open_df = pd.read_table(file, sep = "\t", names=['Index', 'Word', 'POS Tag'], skip_blank_lines=False)
        else:
            file =  file_path + file_name
            open_df = pd.read_table(file, sep = "\t", names=['Index', 'Word'], skip_blank_lines=False)
        
    return open_df

In [3]:
def update_df_rows_with_dummy(df: pd.DataFrame, new_columns_name: list) -> pd.DataFrame:  
    """Update the rows of the dataframe if blank space, fill with dummy"""  

    dummy_row = pd.DataFrame([['0.0', ' ', 'dummy']], columns=df.columns)
    df = pd.concat([dummy_row, df], ignore_index=True)
    df.columns = new_columns_name
    df.fillna("dummy", inplace=True)
   
    return df

In [4]:
train_df = load_data('data/', 'train', False)
dev_df = load_data('data/', 'dev', False)
test_df = load_data('data/', 'test', True)

In [5]:
train_dev_columns_name = ['Index', 'Word', 'POS Tag']

updated_train_df = update_df_rows_with_dummy(train_df, train_dev_columns_name)
updated_dev_df = update_df_rows_with_dummy(dev_df, train_dev_columns_name)

In [6]:
all_pos_tags = updated_train_df['POS Tag'].unique()
all_pos_tags

array(['dummy', 'NNP', ',', 'CD', 'NNS', 'JJ', 'MD', 'VB', 'DT', 'NN',
       'IN', '.', 'VBZ', 'VBG', 'CC', 'VBD', 'VBN', 'RB', 'TO', 'PRP',
       'RBR', 'WDT', 'VBP', 'RP', 'PRP$', 'JJS', 'POS', '``', 'EX', "''",
       'WP', ':', 'JJR', 'WRB', '$', 'NNPS', 'WP$', '-LRB-', '-RRB-',
       'PDT', 'RBS', 'FW', 'UH', 'SYM', 'LS', '#'], dtype=object)

# Outline of Tasks

1. Vocabulary Creation
2. Model Learning
3. Greedy Decoding with HMM
4. Viterbi Decoding with HMM


# 1. Vocabulary Creation

- **Problem:** Creating vocabulary to handle unkown words.
    - **Solution:** Replace rare words wtih whose occurrences are less than a threshold (ie: 3) with a special token `< unk >`

---

1. [x] Create a vocabulary using the training data in the file train
2. [x] Output the vocabulary into a txt file named `vocab.txt`
    - [x] See PDF on how to properly format vocabulary file
3. [x] Questions
    1. [x] What is the selected threshold for unknown words replacement? 3
    2. [x] What is the total size of your vocabulary? 13751
    3. [x] What is the total occurrences of the special token `< unk >` after replacement? 29443

In [7]:
true_false_series = updated_train_df['Word'].value_counts()
vocab_df = pd.DataFrame(true_false_series)
vocab_df.reset_index(inplace = True)
vocab_df

,Word,count
0,",",46476
1,the,39533
2,dummy,38234
3,.,37452
4,of,22104
...,...,...
43188,Birthday,1
43189,Happy,1
43190,Bertie,1
43191,crouched,1


In [8]:
def create_vocab_threshold_df(df: pd.DataFrame, word_col_name: str, count_col_name: str, threhold: int, special_token: str, save_df: bool, save_path_with_name: str):
    """For every word in df, replace with special_token if below threshold
    """
    true_false_series = df[count_col_name] > 3
    
    updated_vocab_df = df.loc[true_false_series == True]
    updated_false_vocab_df = df.loc[true_false_series == False]
    updated_false_vocab_df[word_col_name] = special_token
    
    N_updated_false_vocab_df = len(updated_false_vocab_df)
    
    new_row = pd.DataFrame([[special_token, N_updated_false_vocab_df]], columns=updated_vocab_df.columns)
    final_df = pd.concat([new_row, updated_vocab_df], ignore_index=True)
    N_vocab = range(0, len(updated_vocab_df)+1)
    
    final_df["index"] = N_vocab
    
    final_df = final_df.reindex(columns=[word_col_name, "index", count_col_name])
    if save_df == True:
        final_df.to_csv(save_path_with_name, header=None, index=None, sep='\t')
    
    return final_df

In [31]:
word_col_name = "Word"
count_col_name = "count"
special_token = "< unk >"
save_df = True
save_file_path_and_name = "final_submit/vocab.txt"
updated_vocab_df = create_vocab_threshold_df(vocab_df, word_col_name, count_col_name, 3, special_token, save_df, save_file_path_and_name)

/var/folders/fz/zn5r8vq12nv5p23dtlr15sk40000gn/T/ipykernel_64887/2333569920.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  updated_false_vocab_df[word_col_name] = special_token


In [32]:
updated_vocab_df

,Word,index,count
0,< unk >,0,29443
1,",",1,46476
2,the,2,39533
3,dummy,3,38234
4,.,4,37452
...,...,...,...
13746,trafficking,13746,4
13747,7.62,13747,4
13748,gut,13748,4
13749,17.3,13749,4


# 2. Model Learning

- Learn an HMM from the training data
- **HMM Parameters:**
  <div style="text-align: center;">

    $
    \text{Transition Probability (} t \text{)}: \quad t(s' \mid s) = \frac{\text{count}(s \rightarrow s')}{\text{count}(s)}
    $

    $
    \text{Emission Probability (} e \text{)}: \quad e(x \mid s) = \frac{\text{count}(s \rightarrow x)}{\text{count}(s)}
    $

  </div>

---

1. [x] Learn a model using the training data in the file train
2. [x] Output the learned model into a model file in json format, named `hmm.json`. The model file should contains two dictionaries for the emission and transition parameters, respectively.
    1. [x] 1st dictionary: Named transition, contains items with pairs of (s, s′) as key and t(s′|s) as value. 
    2. [x] 2nd dictionary: Named emission, contains items with pairs of (s, x) as key and e(x|s) as value.
3. Question
    1. [x] How many transition and emission parameters in your HMM? transition = 1416. emission = 50287


In [11]:
def get_counts(df: pd.DataFrame, word_col_name: str, pos_tag_col_name: str, prev_pos_tag_col_name: str):
    """Count the transition and emission states, respectively
    
    Parameters
    ----------
    df: `pd.DataFrame`
        The df to get the words and POS Tags from

    word_col_name: `str`
        The name of the word column in the df

    pos_tag_col_name: `str`
        The name of the POS Tag column in the df
        
    prev_pos_tag_col_name: `str`
        The name of the Previous POS Tag column in the df

    Return
    ------
    transition_states (`dict`), emission_state_word (`dict`), N_state (`dict`): `tuple`
        A tuple with the counts for transition previous state and current state,
        emission state and word, and total number of states
    
    """
    transition_states = defaultdict(int)
    emission_state_word = defaultdict(int)
    N_state = defaultdict(int)
    
    df[prev_pos_tag_col_name] = df[pos_tag_col_name].shift(1) # create new col to store previous states

    # iterate through vocabulary
    for _, row in tqdm(df.iterrows(), total=df.shape[0]):

        emission_state_word[(row[pos_tag_col_name], row[word_col_name])] += 1 # get emissions count at POS Tag col and Word col
        # transition count + 1
        if pd.notnull(row[prev_pos_tag_col_name]):  # Check if it's not NaN
            transition_states[(row[prev_pos_tag_col_name], row[pos_tag_col_name])] += 1 # get transition count at Previous POS Tag col and POS Tag col

        
        N_state[(row[pos_tag_col_name])] += 1 # increment POS Tag to get total number of states (POS Tags)

    return transition_states, emission_state_word, N_state

In [12]:
word_col_name = "Word"
pos_tag_col_name = "POS Tag"
prev_pos_tag_col_name = 'Previous_POS Tag'
transitions, emissions, N_states = get_counts(updated_train_df, word_col_name, pos_tag_col_name, prev_pos_tag_col_name)

100%|██████████| 950313/950313 [00:41<00:00, 22717.16it/s]


In [39]:
print(f"# Transition params = {len(transitions.items())} \n# Emissions params = {len(emissions.items())}")


# Transition params = 1416 
# Emissions params = 50287


In [14]:
def calculate_prob(transitions: dict, emissions: dict, N_states: dict, prob_type: str):   
    """Calculate the transistion and emissions probabilities, respectively

    Parameters
    ----------
    transitions: `dict`
        Counts for transition previous state and current state as key and value as total number (or counts) of pairs
        
    emissions: `dict`
        Counts for emission state and word as key and value as total number (or counts) of pairs

    N_states: `dict`
        Counts of state (POS Tag) as key and value as total number (or counts) of states

    prob_type: `str`
        A string representing either transistion or emissions

    Return
    ------
    store_probs: `dict`
        A dictionary containing the probabilities of transitions and emissions, respectively. Key are pairings and values are probability, respectively
    """

    if prob_type == "t":
        t_or_e = transitions
    elif prob_type == "e":
        t_or_e = emissions
    else:
        print(f"Invalid prob_type {prob_type}")

    store_probs = {}
    for key, value in t_or_e.items():
        
        curr_state = key[0]       
        store_probs[key] = value / N_states[curr_state]
        
    return store_probs

In [15]:
t_probs = calculate_prob(transitions, emissions, N_states, 't')
e_probs = calculate_prob(transitions, emissions, N_states, 'e')

In [16]:
list(t_probs.items())[:7]

[(('dummy', 'NNP'), 0.19789104610393007),
 (('NNP', 'NNP'), 0.3782645420509543),
 (('NNP', ','), 0.13846908958086018),
 ((',', 'CD'), 0.021234939759036144),
 (('CD', 'NNS'), 0.15775891730703062),
 (('NNS', 'JJ'), 0.017196978862406887),
 (('JJ', ','), 0.029129343105320303)]

In [17]:
list(e_probs.items())[:7]

[(('dummy', ' '), 2.6165681092678842e-05),
 (('NNP', 'Pierre'), 6.84868961738654e-05),
 (('NNP', 'Vinken'), 2.2828965391288468e-05),
 ((',', ','), 0.9999139414802065),
 (('CD', '61'), 0.0007168253240050465),
 (('NNS', 'years'), 0.019530237301024905),
 (('JJ', 'old'), 0.003613599348534202)]

### Save HMM Results

In [18]:
save_hmm = "final_submit/hmm.json"

combine_t_and_e_probs = {}
combine_t_and_e_probs["transitions"] = t_probs
combine_t_and_e_probs["emissions"] = e_probs

t_e_probs_df = pd.DataFrame(combine_t_and_e_probs)
t_e_probs_df.to_json(save_hmm) # save

# 3. Greedy Decoding with HMM

1. [x] Implement the greedy decoding algorithm
2. [x] Evaluate it on the development data
3. [x] Predicting the POS Tags of the sentences in the test data
4. [x] Output the predictions in a file named `greedy.out`, in the same format of training data
5. [x] Evaluate the results of the model on `eval.py` in the terminal with `python eval.py − p {predicted file} − g {gold-standard file}`
    - Spefically: `python eval.py -p final_submit/greedy.out -g data/dev`
6. [x] Question
    1. [x] What is the accuracy on the dev data? 80.99%. Possibly need to properly clean data, improve Parts 1 and 2, and include more training data to improve accuracy. I didn’t replace any words based on a certain threshold because I thought it was only for part 1. Some pairs (of both transition and emission, respectively) weren’t found, so I used a low number instead such that we don’t pick that pair. I also need to learn how to write correct and efficient code. Seeing your solution to this HW and previous HWs will help as I struggled on all HWs thus far.

In [19]:
# updated_dev_df.head(40)

In [40]:
def greedy_decoding(dev_df: pd.DataFrame, t_probs: dict, e_probs: dict, N_pos_tags: np.array):
    """Implement greedy decoding on the development file (words only) using the transition probability and emission probability. Furthermore, don't use POS Tag of development file, thus only use POS Tag from training data.

    If 't_' or 'e_', transition and emission probabilities, respectively.

    Parameters
    ----------
    df: `pd.DataFrame`
        Dev file

    t_probs: `py dict`
        Tranision probabilities for POS Tag given previous POS Tag

    e_probs: `py dict`
        Emission probabilities for Word given POS Tags

    N_pos_tags: `np.array`
        All POS Tags found in the training file
    
    Return
    ------
    all_words_with_pos_tag: `list` 
        Store the words with highest probability POS Tag for that word as a tuple
    """

    previous_pos_tag = "dummy"
    not_found_value = 0.000000001
    all_words_with_pos_tag = []

    # Go through each row (word), get the corresponding POS Tag to calculate probabilities    
    for index, row in tqdm(dev_df.iterrows(), total=dev_df.shape[0]):
        # print("index", index, "with word", row['Word'])

        if row['POS Tag'] != "dummy": # check if POS Tag is dummy so we know where each new sentence starts

            # For current word, store score from greedy calculatons. Empty when new word is encountered
            store_scores = []
            
            for N_pos_tags_idx in range(len(N_pos_tags)):
                current_pos_tag = N_pos_tags[N_pos_tags_idx]
    
                """Transition Probability
                    t(t_find_pos_tag | t_given_pos_tag)
                """
                t_find_pos_tag = current_pos_tag
                t_given_pos_tag = previous_pos_tag
                
                """Emission Probability
                    e(e_word | e_given_pos_tag)
                """
                e_word = row['Word']
                e_given_pos_tag = current_pos_tag
                
                """Transition * Emission"""
                t_key = (t_find_pos_tag, t_given_pos_tag)
                e_key = (e_given_pos_tag, e_word)
    
                # IF-ELSE bc not all pairs will be found. If pair is found, use score, otherwise (pair isn't found) set alternative score
                if t_key in t_probs and e_key in e_probs:
                    t = t_probs[t_key]
                    e = e_probs[e_key]
                    score = t * e
                    # print(f"---  t({t_find_pos_tag} | {t_given_pos_tag}) * e({e_word} | {e_given_pos_tag}) = {score}")
                    
                else:
                    t = not_found_value
                    e = not_found_value
                    score = t * e
                    # print(f"--- t({t_find_pos_tag} | {t_given_pos_tag}) * e({e_word} | {e_given_pos_tag}) = {score}")
                            
                store_scores.append(score)
        
            max_score_idx = np.argmax(np.array(store_scores)) # use argmax to get the index of max score
            current_pos_tag = N_pos_tags[max_score_idx] # use the index of the max score to find which POS Tag to 
            all_words_with_pos_tag.append([row['Word'], current_pos_tag]) # store word and POS Tag with max score
            previous_pos_tag = current_pos_tag # update the previous POS Tag
        else:
            empty = "" # formatting final 2D list
            all_words_with_pos_tag.append([empty, empty]) # Adds extra space in final 2D list
        
    return all_words_with_pos_tag

In [41]:
gd_output = greedy_decoding(updated_dev_df, t_probs, e_probs, all_pos_tags)

100%|██████████| 137295/137295 [00:18<00:00, 7353.73it/s]


In [42]:
gd_output = gd_output[1:] # remove intial empty list
# gd_output

### Save Greedy Decoding Results

In [43]:
with open('final_submit/greedy.out', 'w') as op:
    
    index = 1
    for idx, word in enumerate(gd_output):
        if word[0] == "":
            index = 1
            op.write("\n")
        else:
            op.write(f'{index}\t{word[0]}\t{word[1]}')
            op.write("\n")
            index += 1

# 4. Viterbi Decoding with HMM

1. [x] Implement the viterbi decoding algorithm
2. [x] Evaluate it on the development data
3. [x] Predict the POS Tags of the sentences in the test data
4. [x] Output the predictions in a file named `viterbi.out`, in the same format of training data
    - Specifically, `python eval.py -p final_submit/viterbi.out -g data/dev`
5. [x] Question
    1. [x] What is the accuracy on the dev data? 85.27%. Possibly need to properly clean data, improve Parts 1 and 2, and include more training data to improve accuracy. I didn’t replace any words based on a certain threshold because I thought it was only for part 1. Some pairs (of both transition and emission, respectively) weren’t found, so I used a low number instead such that we don’t pick that pair. I also need to learn how to write correct and efficient code. Seeing your solution to this HW and previous HWs will help as I struggled on all HWs thus far.

In [44]:
# Reformat dev df so Viterbi will be more optimized compared to if dev df was a DF
def dataframe_to_list(df: pd.DataFrame):
    """Convert a DF to a list of lists"""
    list_of_sentences = []
    sublist = []

    for _, row in df.iterrows():

        if row['POS Tag'] == 'dummy': # dummy POS Tag indicates a new sentence
            list_of_sentences.append(sublist)
            sublist = []
        else:
            sublist.append(row['Word'])
            
    # Append the last sublist
    if sublist:
        list_of_sentences.append(sublist)
        
    return list_of_sentences


In [45]:
sentences = dataframe_to_list(updated_dev_df)

In [46]:
# sentences[:3]

In [51]:
def viterbi_decoding(sentences: list, t_probs: dict, e_probs: dict, pos_tags: np.array):
    """Implement Viterbi decoding on the development file (words only) using the transition probability and emission probability. 
    
    Parameters
    ----------        
    sentences: `list`
        List of sentences from dev file

    t_probs: `py dict`
        Tranision probabilities for POS Tag given previous POS Tag

    e_probs: `py dict`
        Emission probabilities for Word given POS Tags

    pos_tags: `np.array`
        All POS Tags found in the training file
    
    Return
    ------
    all_words_with_pos_tag: `list` 
        Store the words with highest probability POS Tag for that word as a tuple
    """

    """Clarifications of variables
        - If 't_' or 'e_', transition and emission probabilities, respectively.
        - If `v_pi`, viterbi_pi (from slide deck as it had the pi symbol)"""
    """
    Initialization with base cases
        - For the first word of every new sentence, create a base case
    """
    
    initial_pos_tag = "dummy"
    not_found_value = 0.000001
    all_words_with_pos_tag = []
    
    for sentences_idx in range(len(sentences)):
        sentence = sentences[sentences_idx]
        # print(f"Sentence --- {sentence}")

        store_initial_scores = []

        len_of_sentence = len(sentence)
        N_pos_tags = len(pos_tags)
        
        v_pi = np.zeros((N_pos_tags, len_of_sentence)) # 2D matrix (or table) containing all POS tags and length of each specific sentence

        for pos_tags_idx in range(N_pos_tags):
            initial_t_given_pos_tag = initial_pos_tag
            initial_t_find_pos_tag = pos_tags[pos_tags_idx]
            initial_t_key = (initial_t_find_pos_tag, initial_t_given_pos_tag)
            
            initial_e_given_pos_tag = pos_tags[pos_tags_idx]
            initial_e_word = sentence[0]
            initial_e_key = (initial_e_given_pos_tag, initial_e_word)
            
            # Check if the keys for t_prob and e_prob are valid, respectively. If not, assign alternate score
            if initial_t_key in t_probs and initial_e_key in e_probs:
                v_pi[pos_tags_idx, 0] = t_probs[initial_t_key] * e_probs[initial_e_key]
            else: 
                v_pi[pos_tags_idx, 0] = not_found_value
        
            store_initial_scores.append(v_pi[pos_tags_idx, 0])        
        all_words_with_pos_tag.append([initial_e_word, pos_tags[pos_tags_idx]])


        """DP Algo
            - End base case at first word this sentence
            - For the remaining words in this sentence, find the best combo of word and POS Tag
        """
        previous_word_idx = 0
        
        for word_idx in range(1, len_of_sentence):
            e_word = sentence[word_idx]

            word_with_best_pos_tags = []
            
            for pos_tags_idx in range(N_pos_tags):
                current_pos_tag = pos_tags[pos_tags_idx]
                
                store_scores = []
                
                for previous_pos_tags_idx in range(N_pos_tags):
                    previous_pos_tag = pos_tags[previous_pos_tags_idx]
                    
                    v_pi_idx = (previous_pos_tags_idx, previous_word_idx)
                    """Transition Probability
                        t(t_find_pos_tag | t_given_pos_tag)
                    """
                    t_find_pos_tag = current_pos_tag
                    t_given_pos_tag = previous_pos_tag
                    
                    
                    """Emission Probability
                        e(e_word | e_given_pos_tag)
                    """
                    # e_word is above
                    e_given_pos_tag = current_pos_tag 
                    
                    """Transition * Emission"""
                    t_key = (t_find_pos_tag, t_given_pos_tag)
                    e_key = (e_given_pos_tag, e_word)
                    
                    # IF-ELSE bc not all pairs will be found. If pair is found, use score, otherwise (pair isn't found) set alternative score
                    if t_key in t_probs and e_key in e_probs:
                        t = t_probs[t_key]
                        e = e_probs[e_key]
                        score = v_pi[v_pi_idx] * t * e
                        # print(f"--- FOUND: v_pi[{t_find_pos_tag}, {e_word}] = v_pi[{previous_pos_tag}, {previous_word}] * t({t_find_pos_tag} | {t_given_pos_tag}) * e({e_word} | {e_given_pos_tag}) = {score}")

                    else:
                        t = not_found_value
                        e = not_found_value
                        score = v_pi[v_pi_idx] * t * e
                        # print(f"--- NOT FOUND: v_pi[{t_find_pos_tag}, {e_word}] = v_pi[{previous_pos_tag}, {previous_word}] * t({t_find_pos_tag} | {t_given_pos_tag}) * e({e_word} | {e_given_pos_tag}) = {score}")

                    store_scores.append(score)

                max_score_idx = np.argmax(np.array(store_scores)) # use argmax to get the index of max score
                current_pos_tag = pos_tags[max_score_idx] # use the index of the max score to find which POS Tag to update to
                word_with_best_pos_tags.append(store_scores[max_score_idx]) # store max score 
            
            max_score_of_word_idx = np.argmax(np.array(word_with_best_pos_tags))
            all_words_with_pos_tag.append([e_word, pos_tags[max_score_of_word_idx]])
            
        empty = "" # formatting final 2D list
        all_words_with_pos_tag.append([empty, empty]) # Adds extra space in final 2D list

    return all_words_with_pos_tag

In [52]:
# ignore first sentence as it's empty
# ignore first tag as it's "dummmy"
vd_output = viterbi_decoding(sentences[1:], t_probs, e_probs, all_pos_tags[1:])


In [ ]:
# vd_output

### Save Viterbi Decoding Results

In [ ]:
with open('final_submit/viterbi.out', 'w') as op:
    # # # # # # # 
    index = 1
    for idx, word in enumerate(vd_output):
        if word[0] == "":
            index = 1
            op.write("\n")
        else:
            op.write(f'{index}\t{word[0]}\t{word[1]}')
            op.write("\n")
            index += 1